In [5]:
import experiment_framework as ef
import fast_model_improved as fmi
from pyexpat import model
import networkx as nx
import matplotlib.pyplot as plt
import copy
import numpy as np
import pickle

In [6]:
# Load the graph from the file
save_path = '/Users/huberyhu/Desktop/SURP/im_deinfluence/network_data/as-caida.gpickle'
try:
    with open(save_path, 'rb') as f:
        G = pickle.load(f)
    print(f"Graph loaded from {save_path}")
except Exception as e:
    print(f"An error occurred while loading the graph: {e}")


Graph loaded from /Users/huberyhu/Desktop/SURP/im_deinfluence/network_data/ca-GrQc.gpickle


In [7]:
model = ef.run_influence_cascade(G, num_influencers=50, steps=3, selection_method='random')

# Evaluate the influence and deinfluence
num_influenced = model.evaluate_influence()
num_deinfluenced = model.evaluate_deinfluence()
num_susceptible = model.evaluate_susceptible()

# Create a deep copy of the model
model_fixed_influencers = copy.deepcopy(model)

# Evaluate the influence and deinfluence
num_influenced = model_fixed_influencers.evaluate_influence()
num_deinfluenced = model_fixed_influencers.evaluate_deinfluence()
num_susceptible = model_fixed_influencers.evaluate_susceptible()

print(f"Number of influenced nodes: {num_influenced}")
print(f"Number of deinfluenced nodes: {num_deinfluenced}")
print(f"Number of susceptible nodes: {num_susceptible}")
print('influencer', model.selected_influencers)

Number of influenced nodes: 1362
Number of deinfluenced nodes: 0
Number of susceptible nodes: 3880
influencer {15616, 1025, 13060, 8711, 3209, 7689, 20489, 13712, 8854, 21404, 3999, 8868, 4261, 12453, 19111, 3239, 1958, 6576, 2352, 17585, 25910, 17592, 824, 9913, 18235, 20792, 25661, 2622, 3651, 9413, 23110, 8134, 2504, 22728, 12618, 23240, 13008, 12884, 3032, 20569, 3033, 5083, 25565, 22369, 9189, 1386, 20331, 3821, 19578, 25086}


In [8]:
# Run simulation for different values of k
k_deinfluencers_ls = [0,1,2,3,4,5,6,7,8,9,10]  # Different values of k to test
deinfluencers_list = ef.select_deinfluencers(k_deinfluencers_ls, model_fixed_influencers)
results = ef.average_results(deinfluencers_list, model_fixed_influencers, num_runs=3, steps=2)

# Plotting results
fig, axs = plt.subplots(2, figsize=(10, 10))
axs[0].set_title('Measured Influence against Deinfluencer selection methods')
axs[1].set_title('Measured Deinfluence against Deinfluencer selection methods')

# Create line plots
methods = results[next(iter(results))].keys()  # Get all methods from the first key
k_values = sorted(results.keys())  # Sort k values for plotting

for method in methods:
    influenced_nodes = [results[k][method][0] for k in k_values]
    total_steps = [results[k][method][1] for k in k_values]

    axs[0].plot(k_values, influenced_nodes, label=method, marker='o')
    axs[1].plot(k_values, total_steps, label=method, marker='o')

axs[0].legend()
axs[0].set_xlabel('Number of Deinfluencers')
axs[0].set_ylabel('Average Number of DeInfluenced Nodes')

axs[1].legend()
axs[1].set_xlabel('Number of Deinfluencers')
axs[1].set_ylabel('Average Number of Influenced Nodes')

plt.tight_layout()
plt.show()
